In [1]:
import os
import numpy as np
import pandas as pd
from utils import model_utils

from signlens.params import *
from signlens.preprocessing.data import load_data_subset_csv, load_glossary, load_video_list_json, load_landmarks_json
from signlens.preprocessing.preprocess import group_pad_sequences, encode_labels, decode_labels, pad_and_preprocess_sequence, reshape_processed_data_to_tf
from utils.model_utils import load_model
from utils.video_utils import process_video_to_landmarks_json, draw_landmarks_on_image

%load_ext autoreload

%autoreload 2


2024-03-24 13:38:03.350048: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-24 13:38:04.231850: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
test_data = load_data_subset_csv(balanced=True, csv_path=TRAIN_TEST_CSV_PATH)

Loading data subset from train_test.csv
    ℹ️ Filtered sequences with missing frames. Size reduced from 17233 to 17233 (100.00%)
    ℹ️ Filtered on n_frames = 100. Size reduced from 17233 to 17233 (100.00%)
    ℹ️ Filtered on n_classes = 10. Size reduced from 17233 to 690 (4.00%)
    ℹ️ Balanced data, with average of 34.5 elements per class. Size reduced from 690 to 345 (50.00%)
✅ Loaded 345 rows (2.00% of the original 17233 rows) from the dataset.


In [3]:
test_data

,path,participant_id,sequence_id,sign,n_frames,n_frames2,file_path
0,train_landmark_files_noface/18796/956371192.pa...,18796,956371192,who,56,56,/home/bfrisque/code/benoitfrisque/signlens/raw...
1,train_landmark_files_noface/25571/1433342220.p...,25571,1433342220,all,6,6,/home/bfrisque/code/benoitfrisque/signlens/raw...
2,train_landmark_files_noface/28656/4080345076.p...,28656,4080345076,fine,6,6,/home/bfrisque/code/benoitfrisque/signlens/raw...
3,train_landmark_files_noface/28656/1826108619.p...,28656,1826108619,yes,25,25,/home/bfrisque/code/benoitfrisque/signlens/raw...
4,train_landmark_files_noface/62590/2077886254.p...,62590,2077886254,drink,22,22,/home/bfrisque/code/benoitfrisque/signlens/raw...
...,...,...,...,...,...,...,...
340,train_landmark_files_noface/29302/2740867854.p...,29302,2740867854,fine,9,9,/home/bfrisque/code/benoitfrisque/signlens/raw...
341,train_landmark_files_noface/55372/907853337.pa...,55372,907853337,go,14,14,/home/bfrisque/code/benoitfrisque/signlens/raw...
342,train_landmark_files_noface/36257/2258763216.p...,36257,2258763216,before,68,68,/home/bfrisque/code/benoitfrisque/signlens/raw...
343,train_landmark_files_noface/18796/3734310935.p...,18796,3734310935,no,49,49,/home/bfrisque/code/benoitfrisque/signlens/raw...


# Model predict

In [4]:
model_name = "model 20240322-173411"
model, _ = load_model(model_name)

processed_data = preprocess_and_pad_sequences_from_pq_list(test_data.file_path.iloc[0:2])

prediction = model.predict([processed_data])

word, proba = decode_labels(prediction)

word, proba



Load latest model from local registry...

Load latest model from disk...
✅ Model loaded from local disk /home/bfrisque/code/benoitfrisque/signlens/training_outputs/model 20240322-173411


2024-03-24 13:38:06.985278: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-24 13:38:06.985744: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
/home/bfrisque/.pyenv/versions/3.10.6/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 275ms/step


(['who', 'who'], array([0.45869172, 0.20116639], dtype=float32))

# Video conversion

In [5]:
videos = load_video_list_json()
videos

,sign,bbox,fps,frame_end,frame_start,instance_id,signer_id,source,split,url,variation_id,video_id,video_path
0,airplane,"[605, 21, 1721, 1076]",25,-1,1,10,4,signschool,train,https://signstock.blob.core.windows.net/signsc...,0,01726,/home/bfrisque/code/benoitfrisque/signlens/raw...
1,table,"[374, 52, 810, 720]",25,-1,1,13,38,startasl,train,https://s3-us-west-1.amazonaws.com/files.start...,0,56556,/home/bfrisque/code/benoitfrisque/signlens/raw...
2,see,"[85, 15, 230, 192]",25,-1,1,8,11,signingsavvy,train,https://www.signingsavvy.com/signs/mp4/8/8396.mp4,0,50125,/home/bfrisque/code/benoitfrisque/signlens/raw...
3,who,"[165, 4, 472, 370]",25,-1,1,14,88,aslsignbank,train,https://aslsignbank.haskins.yale.edu/dictionar...,0,66778,/home/bfrisque/code/benoitfrisque/signlens/raw...
4,dog,"[417, 61, 834, 720]",25,-1,1,7,38,startasl,train,https://s3-us-west-1.amazonaws.com/files.start...,1,17086,/home/bfrisque/code/benoitfrisque/signlens/raw...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1471,arm,"[205, 37, 489, 370]",25,-1,1,3,89,aslsignbank,val,https://aslsignbank.haskins.yale.edu/dictionar...,0,65094,/home/bfrisque/code/benoitfrisque/signlens/raw...
1472,say,"[104, 0, 528, 480]",25,-1,1,4,13,asldeafined,val,https://media.asldeafined.com/vocabulary/14687...,0,49430,/home/bfrisque/code/benoitfrisque/signlens/raw...
1473,ear,"[296, 36, 879, 720]",25,-1,1,1,118,aslbrick,train,http://aslbricks.org/New/ASL-Videos/ear.mp4,0,69306,/home/bfrisque/code/benoitfrisque/signlens/raw...
1474,closet,"[64, 14, 260, 240]",25,-1,1,6,26,spreadthesign,train,https://media.spreadthesign.com/video/mp4/13/3...,0,11284,/home/bfrisque/code/benoitfrisque/signlens/raw...


In [6]:
video_path = videos[videos.sign == 'book'].video_path.iloc[0]
video_path


'/home/bfrisque/code/benoitfrisque/signlens/raw_data/WLASL/videos/07070.mp4'

In [7]:
json_data = process_video_to_landmarks_json(video_path, json_output=True, show_preview=False, frame_interval=1)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


✅ Landmarks saved to '/home/bfrisque/code/benoitfrisque/signlens/processed_data/landmarks_videos/07070_landmarks.json'


In [8]:
landmarks_json_path = '/home/bfrisque/code/benoitfrisque/signlens/processed_data/landmarks_videos/07070_landmarks.json'

landmarks = load_landmarks_json(landmarks_json_path)
data_processed = pad_and_preprocess_sequence(landmarks)
data_tf = reshape_processed_data_to_tf(data_processed)
data_tf.shape

TensorShape([1, 100, 225])

In [9]:
model_name = "model 20240322-173411"
model, _ = load_model(model_name)

prediction = model.predict(data_tf)

word, proba = decode_labels(prediction)

word, proba



Load latest model from local registry...

Load latest model from disk...
✅ Model loaded from local disk /home/bfrisque/code/benoitfrisque/signlens/training_outputs/model 20240322-173411
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 256ms/step


(['fine'], array([0.15782131], dtype=float32))

## Write landmarks on video

### Previous version of mediapipe (used in this project)

In [10]:
json_data = process_video_to_landmarks_json(video_path, show_preview=True, save_annotated_video=True)

✅ Landmarks saved to '/home/bfrisque/code/benoitfrisque/signlens/processed_data/landmarks_videos/07070_landmarks.json'
✅ Annotated video saved to '/home/bfrisque/code/benoitfrisque/signlens/processed_data/landmarks_videos/07070_annotated.mp4'


### New version of mediapipe - hand (not used)

In [11]:
# !wget -q https://storage.googleapis.com/mediapipe-models/hand_landmarker/hand_landmarker/float16/1/hand_landmarker.task

In [12]:
# #@markdown We implemented some functions to visualize the hand landmark detection results. <br/> Run the following cell to activate the functions.

# from mediapipe import solutions
# from mediapipe.framework.formats import landmark_pb2
# import numpy as np

# MARGIN = 10  # pixels
# FONT_SIZE = 1
# FONT_THICKNESS = 1
# HANDEDNESS_TEXT_COLOR = (88, 205, 54) # vibrant green

# def draw_landmarks_on_image_new(rgb_image, detection_result):
#   hand_landmarks_list = detection_result.hand_landmarks
#   handedness_list = detection_result.handedness
#   annotated_image = np.copy(rgb_image)

#   # Loop through the detected hands to visualize.
#   for idx in range(len(hand_landmarks_list)):
#     hand_landmarks = hand_landmarks_list[idx]
#     handedness = handedness_list[idx]

#     # Draw the hand landmarks.
#     hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
#     hand_landmarks_proto.landmark.extend([
#       landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks
#     ])
    
#     solutions.drawing_utils.draw_landmarks(
#       annotated_image,
#       hand_landmarks_proto,
#       solutions.hands.HAND_CONNECTIONS,
#       solutions.drawing_styles.get_default_hand_landmarks_style(),
#       solutions.drawing_styles.get_default_hand_connections_style())

#     # Get the top left corner of the detected hand's bounding box.
#     height, width, _ = annotated_image.shape
#     x_coordinates = [landmark.x for landmark in hand_landmarks]
#     y_coordinates = [landmark.y for landmark in hand_landmarks]
#     text_x = int(min(x_coordinates) * width)
#     text_y = int(min(y_coordinates) * height) - MARGIN

#     # Draw handedness (left or right hand) on the image.
#     cv2.putText(annotated_image, f"{handedness[0].category_name}",
#                 (text_x, text_y), cv2.FONT_HERSHEY_DUPLEX,
#                 FONT_SIZE, HANDEDNESS_TEXT_COLOR, FONT_THICKNESS, cv2.LINE_AA)

#   return annotated_image

In [13]:
# import cv2
# import numpy as np
# import mediapipe as mp
# from mediapipe import solutions
# from mediapipe.framework.formats import landmark_pb2
# from mediapipe.tasks import python
# from mediapipe.tasks.python import vision

# base_options = python.BaseOptions(model_asset_path='hand_landmarker.task')
# running_mode =  vision.RunningMode('VIDEO')
# options = vision.HandLandmarkerOptions(base_options=base_options,
#                                        num_hands=2)
# detector = vision.HandLandmarker.create_from_options(options)

# cap = cv2.VideoCapture(video_path)

# # Load the frame rate of the video using OpenCV's CAP_PROP_FPS
# fps = cap.get(cv2.CAP_PROP_FPS)

# while cap.isOpened():
#     ret, frame = cap.read()
#     if not ret:
#         break
    
#     # Convert the frame to RGB
#     rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
#     # Convert the frame to MediaPipe's Image object
#     image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb_frame)
    
#     # Get the timestamp for the current frame
#     frame_timestamp_ms = cap.get(cv2.CAP_PROP_POS_MSEC)

#         # STEP 4: Detect hand landmarks from the input image.
#     detection_result = detector.detect(image)

#     # # STEP 5: Process the classification result. In this case, visualize it.
#     annotated_image = draw_landmarks_on_image_new(image.numpy_view(), detection_result)
#     annotated_image_color = cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR)

#     cv2.imshow('Annotated Image', annotated_image_color)
   
    
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#          break

# # Release the video capture object and close all windows
# cap.release()
# cv2.destroyAllWindows()
